In [1]:
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import torch

# Load the model from Hugging Face
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Configure scheduler and move the model to GPU
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/939 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [2]:
# Save the model to a local directory
pipe.save_pretrained("./stable_diffusion_local")

In [3]:
from diffusers import StableDiffusionPipeline
import torch

# Load the model from local directory
pipe = StableDiffusionPipeline.from_pretrained("./stable_diffusion_local", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import gradio as gr

# Clear GPU cache
torch.cuda.empty_cache()

# Load the model
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

# Image generation function
def image_generator(prompt, width=512, height=512):
    """
    Generates an image from the given prompt using Stable Diffusion.

    Parameters:
        prompt (str): The text prompt to generate the image.
        width (int): The width of the output image.
        height (int): The height of the output image.

    Returns:
        PIL.Image: The generated image.
    """
    image = pipe(prompt, width=width, height=height).images[0]
    return image

# Build Gradio interface
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 🌟 AI-Powered Art Generator oleh mhdays🎨
        Masukkan prompt di bawah untuk membuat gambar menakjubkan menggunakan **Stable Diffusion**.
        Sesuaikan dimensi dan saksikan imajinasi Anda menjadi nyata! 🚀
        """
    )

    with gr.Row():
        with gr.Column():
            prompt_input = gr.Textbox(
                label="🎤 Enter Your Prompt",
                placeholder="e.g., A futuristic cityscape with glowing neon lights",
                lines=2,
            )
            width_slider = gr.Slider(
                minimum=256, maximum=1024, value=512, step=64, label="📏 Image Width"
            )
            height_slider = gr.Slider(
                minimum=256, maximum=1024, value=512, step=64, label="📐 Image Height"
            )
            generate_button = gr.Button("✨ Generate Image")

        with gr.Column():
            output_image = gr.Image(label="🎨 Generated Image")

    generate_button.click(
        fn=image_generator,
        inputs=[prompt_input, width_slider, height_slider],
        outputs=output_image,
    )

#Launch the app
demo.launch()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://287dbb2df1e3f30b03.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
